In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import regularizers


2025-05-17 09:27:49.538622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747474069.866231      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747474069.960493      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
import numpy as np
import PIL
import pandas as pd

In [12]:
print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)
print("Pillow version:", PIL.__version__)
print("pandas version", pd.__version__)

TensorFlow version: 2.18.0
NumPy version: 1.26.4
Pillow version: 11.1.0
pandas version 2.2.3


In [ ]:
base_model = MobileNetV2(weights='imagenet', 
                        include_top=False, 
                        input_shape=(224,224,3)
                       
                       )

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(4, activation='softmax')(x)


In [ ]:
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=7,            
    width_shift_range=0.03,      
    height_shift_range=0.03,
    zoom_range=0.08,              
    shear_range=0.03,
    brightness_range=[0.85, 1.15], 
    fill_mode='nearest'          
)

val_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/lungs-dataset/Lungs disease dataset/train',  
    target_size=(224, 224),
    batch_size=20,
    class_mode='categorical',
)

val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/lungs-dataset/Lungs disease dataset/val',
    target_size=(224, 224),
    batch_size=20,
    class_mode='categorical',
)



In [ ]:
x_batch, y_batch = next(train_generator)

# Check the shape
print("Image shape:", x_batch.shape)

# Check pixel range
print("Min pixel value:", x_batch.min())
print("Max pixel value:", x_batch.max())
print("y_batch shape:", y_batch.shape) 

In [ ]:
callbacks = [
    ModelCheckpoint('mobilenetv2_lr1e-7(1)_adam_unfreeze_best.weights.h5',
                    save_best_only=True, 
                    save_weights_only=True,
                    monitor='val_accuracy', 
                    mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, verbose=1),
    

]

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy',color='red')  
plt.plot(history.history['val_accuracy'], label='Validation Accuracy',color='green')  

plt.title('Model Accuracy')  
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')  

plt.legend() 
plt.show() 


In [ ]:
plt.plot(history.history['loss'], label='Training loss')  
plt.plot(history.history['val_loss'], label='Validation loss')  
plt.title('Model loss')  
plt.xlabel('Epochs') 
plt.ylabel('loss')  

plt.legend() 
plt.show() 


In [3]:
# model.save('adam_lr1e-5_7_model.h5')

In [16]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/input/new-model/adam_lr1e-5_7_model.h5')

In [ ]:
# model.summary()

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/lungs-dataset/Lungs disease dataset/test',  
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)


Found 430 images belonging to 4 classes.


In [17]:
loss, accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 309ms/step - accuracy: 0.9475 - loss: 0.1457
Test Loss: 0.1236
Test Accuracy: 95.58%
